## Retail Analytics - Wait time Analyzer

This guide walks us through building a custom analyzer, by extending result analyzer base class for retail analytics. Specifically, this analyzer tracks queue wait time, current people in the queue, average wait time of completed waits

In [ ]:
import time
import cv2
import numpy as np
from collections import deque
from sklearn.metrics.pairwise import cosine_similarity
from degirum_tools.result_analyzer_base import ResultAnalyzerBase
import degirum as dg
import degirum_tools
from collections import defaultdict

## Wait Time Analyzer
We extend ResultAnalyzerBase class to write our custom analyzer logic for tracking queue wait time

In [ ]:
class WaitTimeAnalyzer(ResultAnalyzerBase):
    """
    Analyzer for tracking person with wait time

    Tracks:
    - Current people in zone with their wait times
    - Average wait time of completed waits
    """

    def __init__(self, waiting_zone, config=None):
        """
        Args:
            waiting_zone: np.array of polygon coordinates [[x1,y1], [x2,y2], ...]
            config: Optional configuration dict
        """
        super().__init__()
        self.waiting_zone = np.array(waiting_zone, np.int32)

        # Configuration
        default_config = {
            "person_confidence": 0.35,  # Min confidence for person detection
            "max_disappeared": 30,  # Frames before considering person left
            "tracking_threshold_percent": 10.0,  # Percentage of image width for tracking threshold
        }
        self.config = {**default_config, **(config or {})}

        # Tracking state
        self.tracked_people = {}  # {person_id: PersonTrack}
        self.next_person_id = 1
        self.completed_waits = []  # List of completed wait times
        self.last_5_waits = deque(maxlen=5)

        # Frame management
        self.frame_count = 0
        self.fps = 30.0  # Default, will be updated if available

    def reset(self):
        """Reset all tracking state"""
        self.tracked_people = {}
        self.next_person_id = 1
        self.completed_waits = []
        self.last_5_waits.clear()
        self.frame_count = 0

    def analyze(self, inference_result):
        """Main processing function called by DeGirum for each frame"""
        timestamp = self.frame_count / self.fps
        self.frame_count += 1
        person_detections = [
            det for det in inference_result.results
            if det["label"].lower() == "person" and det["score"] >= self.config["person_confidence"]
        ]
        # Update tracking
        image_width = inference_result.image.shape[1] if hasattr(inference_result, 'image') else 640
        self._update_tracking(person_detections, timestamp, image_width)

        # Add stats to inference result
        stats = self.get_stats()
        stats["timestamp"] = timestamp
        stats["frame_count"] = self.frame_count

        # Add stats as a new result entry
        stats_result = {
            "label": "queue_stats",
            "score": 1.0,
            # "bbox": [0, 0, 0, 0],  # Dummy bbox for stats
            "stats": stats
        }
        inference_result.results.append(stats_result)

        # Annotate frame
        self._draw_annotations(inference_result.image_overlay, timestamp)

        return inference_result

    def annotate(self, inference_result, image):
        """Draw overlays on video frame"""
        timestamp = self.frame_count / self.fps
        self._draw_annotations(image, timestamp)
        return image


    def _match_to_existing_track(self, center, image_width):
        """
        Match detection to existing tracked person using percentage-based

        Args:
            center: (x, y) center point of current detection
            image_width: Width of the image for percentage calculation

        Returns:
            person_id if match found, None otherwise
        """
        if not self.tracked_people:
            return None

        # Calculate threshold distance as percentage of image width
        threshold_distance = (self.config["tracking_threshold_percent"] / 100.0) * image_width

        for person_id, track in self.tracked_people.items():
            # Skip if person disappeared too long ago
            if track["frames_disappeared"] > self.config["max_disappeared"]:
                continue

            # Calculate distance between centers
            track_center = track["center"]
            distance = abs(center[0] - track_center[0]) + abs(center[1] - track_center[1])

            # Match if close enough (within percentage threshold)
            if distance < threshold_distance:
                return person_id

        return None

    def _update_tracking(self, detections, timestamp, image_width=None):
        """Update tracking state for all detections"""

        # Mark all as not seen this frame
        for person_id in self.tracked_people:
            self.tracked_people[person_id]["frames_disappeared"] += 1

        current_frame_ids = set()

        # Process each detection
        for det in detections:
            bbox = det["bbox"]
            center = self._get_center(bbox)
            in_zone = self._point_in_polygon(center, self.waiting_zone)

            # Try to match with existing track
            person_id = self._match_to_existing_track(center, image_width or 640)  # Default to 640 if not provided

            # Create new track if no match
            if person_id is None:
                person_id = self.next_person_id
                self.next_person_id += 1

                self.tracked_people[person_id] = {
                    "enter_time": None,
                    "wait_time": 0.0,
                    "completed": False,
                    "center": center,
                    "bbox": bbox,
                    "frames_disappeared": 0,
                    "in_zone": False,
                }

            # Update track
            track = self.tracked_people[person_id]
            track["center"] = center
            track["bbox"] = bbox
            track["frames_disappeared"] = 0
            current_frame_ids.add(person_id)

            # Zone entry logic
            if in_zone and track["enter_time"] is None:
                track["enter_time"] = timestamp
                track["in_zone"] = True

            # Zone exit logic
            if not in_zone and track["enter_time"] is not None and not track["completed"]:
                wait_time = timestamp - track["enter_time"]
                track["wait_time"] = wait_time
                track["completed"] = True
                track["in_zone"] = False

                self.completed_waits.append(wait_time)
                self.last_5_waits.append(wait_time)

            # Update wait time for people currently in zone
            if in_zone and track["enter_time"] is not None and not track["completed"]:
                track["wait_time"] = timestamp - track["enter_time"]
                track["in_zone"] = True
            elif not in_zone:
                track["in_zone"] = False

        # Remove tracks that have disappeared for too long
        to_remove = []
        for person_id, track in self.tracked_people.items():
            if track["frames_disappeared"] > self.config["max_disappeared"]:
                to_remove.append(person_id)

        for person_id in to_remove:
            del self.tracked_people[person_id]

    def _get_center(self, bbox):
        """Get center point of bounding box"""
        x1, y1, x2, y2 = bbox
        return (int((x1 + x2) / 2), int((y1 + y2) / 2))

    def _point_in_polygon(self, point, polygon):
        """Check if point is inside polygon"""
        return cv2.pointPolygonTest(polygon, point, False) >= 0

    def _draw_annotations(self, frame, timestamp):
        """Draw zone and tracking info on frame"""

        # Draw waiting zone
        cv2.polylines(frame, [self.waiting_zone], True, (0, 0, 255), 2)

        # Draw zone label
        zone_center = np.mean(self.waiting_zone, axis=0).astype(int)
        cv2.putText(frame, "WAITING ZONE",
                   tuple(zone_center - [60, 10]),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Draw each tracked person
        for person_id, track in self.tracked_people.items():
            if track["frames_disappeared"] > 0:
                continue

            bbox = track["bbox"]
            x1, y1, x2, y2 = map(int, bbox)

            # Color based on zone status
            color = (0, 255, 255) if track["in_zone"] else (255, 0, 255)

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            # Draw person ID and wait time
            # label = f"P{person_id}: {track['wait_time']:.1f}s"
            # cv2.putText(frame, label, (x1, y1 - 5),
            #            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            cv2.putText(frame, f"person-{person_id}", (x1 + 80, y1 - 25),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)
            cv2.putText(frame, f"WT: {track['wait_time']:.1f}s", (x1 + 80, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)

    def get_stats(self):
        """
        Get current statistics

        Returns:
            dict with:
                - people_in_zone: {person_id: wait_time}
                - current_ids_in_zone: [person_ids]
                - average_wait_time: float
                - total_processed: int
                - last_5_waits: list
        """
        people_in_zone = {
            pid: round(track["wait_time"], 2)
            for pid, track in self.tracked_people.items()
            if track["in_zone"] and track["frames_disappeared"] == 0
        }

        current_ids = list(people_in_zone.keys())

        avg_wait = (
            round(sum(self.completed_waits) / len(self.completed_waits), 2)
            if self.completed_waits else 0.0
        )

        return {
            "people_in_zone": people_in_zone,
            "current_ids_in_zone": current_ids,
            "average_wait_time": avg_wait,
            "total_processed": len(self.completed_waits),
            "last_5_waits": [round(w, 2) for w in list(self.last_5_waits)],
        }

    def _print_stats(self, timestamp):
        """Print current statistics to console"""
        stats = self.get_stats()

        print("=" * 60)
        print(f"Timestamp: {time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Video Time: {timestamp:.1f}s")
        print(f"\nPeople in Zone: {stats['people_in_zone']}")
        print(f"Current IDs: {stats['current_ids_in_zone']}")
        print(f"Average Wait Time: {stats['average_wait_time']}s")
        print(f"Total Processed: {stats['total_processed']}")
        print(f"Last 5 Waits: {stats['last_5_waits']}")
        print("=" * 60 + "\n")

## Attach analyzer and run inference

In [ ]:
if __name__ == "__main__":

    # Configuration
    inference_host_address = "@local"  # or "@cloud"
    zoo_url = "degirum/hailo"
    device_type = "HAILORT/HAILO8L"
    model_name="yolov8n_relu6_person--640x640_quant_hailort_multidevice_1"
    video_source = 'video.mp4' #replace video source

    # Define waiting zone polygon
    waiting_zones = np.array([
        [200, 150],
        [850, 150],
        [850, 600],
        [200, 600]
    ], np.int32)

    # Load person detection model
    detection_model = dg.load_model(
        model_name=model_name,
        inference_host_address=inference_host_address,
        zoo_url=zoo_url
    )


    # Create analyzer
    wait_time_analyzer = WaitTimeAnalyzer(
        waiting_zone=waiting_zones,
        config={
            "person_confidence": 0.35,
            "tracking_threshold_percent": 10.0,  # 10% of image width for tracking threshold
        }
    )

    # Attach analyzer
    degirum_tools.attach_analyzers(detection_model, [wait_time_analyzer])

    # Run inference
    print("Starting inference...")

    with degirum_tools.Display("Wait Time Tracker") as display:
        for result in degirum_tools.predict_stream(detection_model, video_source):
            display.show(result)

## Print stats from detection results

In [ ]:
detection["stats"]